# Weather Data LSTM & CNN

## This Script Contains the Following:
1. Importing Files and Libraries
2. Preparing Datasets
3. LSTM & CNN Models 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import unique
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D

In [65]:
path = r'/Users/niravbariya/Documents/Data Science/CareerFoundry/Machine Learning Specialization/Achievement 1 & 2/02 Data Sets'

In [66]:
tf.random.set_seed(42)

In [67]:
weather = pd.read_csv(os.path.join(path, 'Unsupervised', 'Dataset-weather-prediction-dataset-processed.csv'), index_col=False)

In [68]:
weather_pred = pd.read_csv(os.path.join(path, 'Unsupervised', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'),index_col=False)

In [69]:
pd.set_option('display.max_columns',None)

In [70]:
weather.head(3)

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_snow_depth,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,GDANSK_cloud_cover,GDANSK_humidity,GDANSK_precipitation,GDANSK_snow_depth,GDANSK_temp_mean,GDANSK_temp_min,GDANSK_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_snow_depth,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_snow_depth,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_snow_depth,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,ROMA_cloud_cover,ROMA_wind_speed,ROMA_humidity,ROMA_pressure,ROMA_sunshine,ROMA_temp_mean,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.0,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,7.7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,5.4,0.83,1.0161,0.12,0.08,0,0.0,10.0,7.0,11.5,8,0.91,0.00,0,0.8,-0.3,1.6,7,0.91,1.0010,0.13,0.22,0,0.0,10.6,9.4,8.3,2.9,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.4,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,8.7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.0,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,0.20,0.1,0,0.0,6.9,1.1,10.4,8,4.0,0.98,0.9978,0.04,1.14,0,0.0,4.9,3.8,5.9,3,2.6,0.73,1.0152,7.1,7.8,4,4.5,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,3.8,0.76,1.0169,1.54,0.44,10.0,7.8,12.2,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1

## 02. Preparing Datasets

In [71]:
# Extracting Date column for combining with cleaned dataframe later on
date_ = weather['DATE'].values

In [72]:
weather = weather.drop(columns = ['DATE', 'MONTH'])

In [73]:
weather.shape

(22950, 168)

In [74]:
weather_pred.columns

Index(['DATE', 'BASEL_pleasant_weather', 'BELGRADE_pleasant_weather',
       'BUDAPEST_pleasant_weather', 'DEBILT_pleasant_weather',
       'DUSSELDORF_pleasant_weather', 'HEATHROW_pleasant_weather',
       'KASSEL_pleasant_weather', 'LJUBLJANA_pleasant_weather',
       'MAASTRICHT_pleasant_weather', 'MADRID_pleasant_weather',
       'MUNCHENB_pleasant_weather', 'OSLO_pleasant_weather',
       'SONNBLICK_pleasant_weather', 'STOCKHOLM_pleasant_weather',
       'VALENTIA_pleasant_weather'],
      dtype='object')

In [75]:
# Removing weather stations with no prediction data
weather = weather.drop(columns = [ 'GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 'GDANSK_snow_depth', 
                                  'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max', 'ROMA_cloud_cover', 
                                  'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 
                                  'ROMA_temp_mean', 'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure', 
                                  'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 
                                  'TOURS_temp_min', 'TOURS_temp_max'])

In [76]:
# Removing observations regarding wind_speed and snow_depth
weather = weather.drop(columns = ['BASEL_wind_speed','BASEL_snow_depth','DEBILT_wind_speed','DUSSELDORF_wind_speed',
                                 'DUSSELDORF_snow_depth','HEATHROW_snow_depth','KASSEL_wind_speed','LJUBLJANA_wind_speed',
                                 'MAASTRICHT_wind_speed','MADRID_wind_speed','MUNCHENB_snow_depth','OSLO_wind_speed',
                                 'OSLO_snow_depth','SONNBLICK_wind_speed','VALENTIA_snow_depth'])

In [77]:
weather.shape

(22950, 132)

In [78]:
#Copying data from Close enough stations:
#Ljubljana is near Kassel, Sonnblick is near Munchen, and Olso is close enough to Stockholm
weather['KASSEL_cloud_cover'] = weather['LJUBLJANA_cloud_cover'].values

In [79]:
weather['MUNCHENB_pressure'] = weather['SONNBLICK_pressure'].values

In [80]:
weather['STOCKHOLM_humidity'] = weather['OSLO_humidity'].values

In [81]:
weather.shape

(22950, 135)

In [84]:
# Reordering columns
columns = ['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
           'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BELGRADE_cloud_cover', 'BELGRADE_humidity',
           'BELGRADE_pressure', 'BELGRADE_global_radiation', 'BELGRADE_precipitation', 'BELGRADE_sunshine', 'BELGRADE_temp_mean',
           'BELGRADE_temp_min', 'BELGRADE_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure',
           'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_min',
           'BUDAPEST_temp_max', 'DEBILT_cloud_cover', 'DEBILT_humidity', 'DEBILT_pressure', 'DEBILT_global_radiation',
           'DEBILT_precipitation', 'DEBILT_sunshine', 'DEBILT_temp_mean', 'DEBILT_temp_min', 'DEBILT_temp_max', 'DUSSELDORF_cloud_cover',
           'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_radiation', 'DUSSELDORF_precipitation', 'DUSSELDORF_sunshine',
           'DUSSELDORF_temp_mean', 'DUSSELDORF_temp_min', 'DUSSELDORF_temp_max', 'HEATHROW_cloud_cover', 'HEATHROW_humidity',
           'HEATHROW_pressure', 'HEATHROW_global_radiation', 'HEATHROW_precipitation', 'HEATHROW_sunshine', 'HEATHROW_temp_mean',
           'HEATHROW_temp_min', 'HEATHROW_temp_max', 'KASSEL_cloud_cover', 'KASSEL_humidity', 'KASSEL_pressure', 'KASSEL_global_radiation',
           'KASSEL_precipitation', 'KASSEL_sunshine', 'KASSEL_temp_mean', 'KASSEL_temp_min', 'KASSEL_temp_max', 'LJUBLJANA_cloud_cover',
           'LJUBLJANA_humidity', 'LJUBLJANA_pressure', 'LJUBLJANA_global_radiation', 'LJUBLJANA_precipitation', 'LJUBLJANA_sunshine',
           'LJUBLJANA_temp_mean', 'LJUBLJANA_temp_min', 'LJUBLJANA_temp_max', 'MAASTRICHT_cloud_cover', 'MAASTRICHT_humidity',
           'MAASTRICHT_pressure', 'MAASTRICHT_global_radiation', 'MAASTRICHT_precipitation', 'MAASTRICHT_sunshine', 'MAASTRICHT_temp_mean',
           'MAASTRICHT_temp_min', 'MAASTRICHT_temp_max', 'MADRID_cloud_cover', 'MADRID_humidity', 'MADRID_pressure', 'MADRID_global_radiation',
           'MADRID_precipitation', 'MADRID_sunshine', 'MADRID_temp_mean', 'MADRID_temp_min', 'MADRID_temp_max', 'MUNCHENB_cloud_cover',
           'MUNCHENB_humidity', 'MUNCHENB_pressure', 'MUNCHENB_global_radiation', 'MUNCHENB_precipitation', 'MUNCHENB_sunshine',
           'MUNCHENB_temp_mean', 'MUNCHENB_temp_min', 'MUNCHENB_temp_max', 'OSLO_cloud_cover', 'OSLO_humidity', 'OSLO_pressure',
           'OSLO_global_radiation', 'OSLO_precipitation', 'OSLO_sunshine', 'OSLO_temp_mean', 'OSLO_temp_min', 'OSLO_temp_max',
           'SONNBLICK_cloud_cover', 'SONNBLICK_humidity', 'SONNBLICK_pressure', 'SONNBLICK_global_radiation', 'SONNBLICK_precipitation',
           'SONNBLICK_sunshine', 'SONNBLICK_temp_mean', 'SONNBLICK_temp_min', 'SONNBLICK_temp_max', 'STOCKHOLM_cloud_cover', 'STOCKHOLM_humidity',
           'STOCKHOLM_pressure', 'STOCKHOLM_global_radiation', 'STOCKHOLM_precipitation', 'STOCKHOLM_sunshine', 'STOCKHOLM_temp_mean',
           'STOCKHOLM_temp_min', 'STOCKHOLM_temp_max', 'VALENTIA_cloud_cover', 'VALENTIA_humidity', 'VALENTIA_pressure', 'VALENTIA_global_radiation',
           'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean', 'VALENTIA_temp_min', 'VALENTIA_temp_max']

In [85]:

weather = weather[columns]

In [86]:
weather.head(2)

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.0,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.1,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.0,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.3,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1


In [87]:
weather_pred = weather_pred.drop(columns = 'DATE')

In [88]:
weather_pred.shape

(22950, 15)

In [89]:
weather_pred.head(2)

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
predictions = {0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT', 4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA',
               8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO', 12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'}

In [540]:
X = weather
y = weather_pred

In [541]:
X = np.array(X.iloc[:])

In [542]:
X = X.reshape(-1,15,9)

In [543]:
X.shape

(22950, 15, 9)

In [544]:
y.shape

(22950, 15)

In [545]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [546]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360, 15)
(4590, 15, 9) (4590, 15)


In [547]:
# Defining count class function
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [625]:
# First Model
epochs = 4
batch_size = 10
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='sigmoid')) 

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [626]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [627]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs,verbose = 1)

Epoch 1/4
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 902us/step - accuracy: 0.0729 - loss: 8.8627 - val_accuracy: 0.0340 - val_loss: 8.6839
Epoch 2/4
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 862us/step - accuracy: 0.0795 - loss: 9.0919 - val_accuracy: 0.0285 - val_loss: 8.7960
Epoch 3/4
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 851us/step - accuracy: 0.1033 - loss: 9.1683 - val_accuracy: 0.0399 - val_loss: 9.0051
Epoch 4/4
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 843us/step - accuracy: 0.0961 - loss: 9.3853 - val_accuracy: 0.0736 - val_loss: 9.2570


The accuracy is extremely low and the loss functions keeps increasing.

In [628]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([predictions[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([predictions[y] for y in np.argmax(y_pred, axis=1)])
    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [629]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
Pred        BASEL  BELGRADE  BUDAPEST  HEATHROW  LJUBLJANA  MADRID
True                                                              
BASEL        1266        16         1         2          0    1696
BELGRADE      794         8         0         0          0      74
BUDAPEST      146         1         0         0          0       8
DEBILT         51         0         0         0          0       0
DUSSELDORF     33         0         0         0          0       1
HEATHROW       65         0         0         0          0       4
KASSEL          9         0         0         0          0       0
LJUBLJANA      49         0         0         3          0       7
MAASTRICHT      4         0         0         0          0       1
MADRID        162         5         0         3          1     168
MUNCHENB        5         0         0         0          0       0
OSLO            4         0         0         0          0       1
STOCKHOLM       1  

In [630]:
# Second Model
epochs = 8
batch_size = 10
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='sigmoid')) 

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [631]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [632]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs,verbose = 1)

Epoch 1/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 886us/step - accuracy: 0.1103 - loss: 9.1764 - val_accuracy: 0.0399 - val_loss: 8.9852
Epoch 2/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 847us/step - accuracy: 0.0818 - loss: 9.7311 - val_accuracy: 0.0460 - val_loss: 9.1940
Epoch 3/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 838us/step - accuracy: 0.0854 - loss: 9.8306 - val_accuracy: 0.0464 - val_loss: 9.3567
Epoch 4/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 840us/step - accuracy: 0.0847 - loss: 9.9282 - val_accuracy: 0.0390 - val_loss: 9.5846
Epoch 5/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 847us/step - accuracy: 0.0766 - loss: 10.0827 - val_accuracy: 0.0739 - val_loss: 9.8890
Epoch 6/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 848us/step - accuracy: 0.0768 - loss: 10.3543 - val_accuracy: 0.0734 - val_loss: 10.2344
Epoch 7/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 862us/step - accuracy: 0.0778 - loss: 10.6505 - val_accuracy: 0.0739 - val_loss: 10.5379
Epoch 8/8
1836/1836 ━━━━━━━━━━━━━━━━━━━━ 2s 862us/step - accuracy: 0.0699 - lo

Accuracy is decreasing and loss is increasing. The model is not converging with increased number of epochs and n_hidden. Let's increase batch size.

In [645]:
# Third Model
epochs = 8
batch_size = 30
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='sigmoid'))

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [646]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [647]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs,verbose = 1)

Epoch 1/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1307 - loss: 8.9339 - val_accuracy: 0.0281 - val_loss: 8.5520
Epoch 2/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 895us/step - accuracy: 0.0842 - loss: 9.1419 - val_accuracy: 0.0318 - val_loss: 8.4801
Epoch 3/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 895us/step - accuracy: 0.0896 - loss: 9.0670 - val_accuracy: 0.0362 - val_loss: 8.4622
Epoch 4/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.0963 - loss: 9.0670 - val_accuracy: 0.0728 - val_loss: 8.4920
Epoch 5/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 934us/step - accuracy: 0.0960 - loss: 9.0938 - val_accuracy: 0.0763 - val_loss: 8.5506
Epoch 6/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 902us/step - accuracy: 0.0931 - loss: 9.1665 - val_accuracy: 0.0767 - val_loss: 8.6397
Epoch 7/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 898us/step - accuracy: 0.0909 - loss: 9.2376 - val_accuracy: 0.0763 - val_loss: 8.7240
Epoch 8/8
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.0844 - loss: 9.3256 - val_accura

Increasing the batch size doesn't increase the accuracy nor does the loss function converge.

In [648]:
# Fourth Model, increasing epochs and n_hidden from 8 to 16
epochs = 16
batch_size = 30
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='sigmoid'))

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [649]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [650]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs,verbose = 1)

Epoch 1/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0714 - loss: 9.4512 - val_accuracy: 0.0512 - val_loss: 8.7438
Epoch 2/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 973us/step - accuracy: 0.0623 - loss: 9.6639 - val_accuracy: 0.0523 - val_loss: 8.5745
Epoch 3/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 981us/step - accuracy: 0.0783 - loss: 9.5593 - val_accuracy: 0.0477 - val_loss: 8.5505
Epoch 4/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 981us/step - accuracy: 0.0888 - loss: 9.5339 - val_accuracy: 0.0556 - val_loss: 8.6077
Epoch 5/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 987us/step - accuracy: 0.0865 - loss: 9.5571 - val_accuracy: 0.0802 - val_loss: 8.7129
Epoch 6/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0923 - loss: 9.6526 - val_accuracy: 0.0739 - val_loss: 8.8196
Epoch 7/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 983us/step - accuracy: 0.0863 - loss: 9.7524 - val_accuracy: 0.0734 - val_loss: 8.9468
Epoch 8/16
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 972us/step - accuracy: 0.0898 - loss: 9.8557 - val_

The Model still doesn't improve after increasing the epochs and n_hidden. The accuracy continues to remain low.

In [651]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step
Pred        BASEL  BELGRADE  MADRID  MUNCHENB
True                                         
BASEL        2973         1       6         1
BELGRADE      872         3       1         0
BUDAPEST      154         1       0         0
DEBILT         51         0       0         0
DUSSELDORF     34         0       0         0
HEATHROW       68         0       1         0
KASSEL          9         0       0         0
LJUBLJANA      59         0       0         0
MAASTRICHT      5         0       0         0
MADRID        333         1       5         0
MUNCHENB        5         0       0         0
OSLO            4         0       1         0
STOCKHOLM       1         0       0         0
VALENTIA        1         0       0         0


### CNN Model

I tried CNN model to check if there is an increase in accuracy or whether loss function converge.

In [652]:
# First CNN Model
epochs = 16
batch_size = 30
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid'))

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [653]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [654]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/16
612/612 - 0s - 782us/step - accuracy: 0.1467 - loss: 1019.0426
Epoch 2/16
612/612 - 0s - 320us/step - accuracy: 0.1275 - loss: 11431.3955
Epoch 3/16
612/612 - 0s - 318us/step - accuracy: 0.1273 - loss: 39656.4297
Epoch 4/16
612/612 - 0s - 324us/step - accuracy: 0.1275 - loss: 89713.7656
Epoch 5/16
612/612 - 0s - 323us/step - accuracy: 0.1281 - loss: 162472.4844
Epoch 6/16
612/612 - 0s - 319us/step - accuracy: 0.1261 - loss: 258070.1719
Epoch 7/16
612/612 - 0s - 319us/step - accuracy: 0.1217 - loss: 375079.0938
Epoch 8/16
612/612 - 0s - 320us/step - accuracy: 0.1164 - loss: 511139.1250
Epoch 9/16
612/612 - 0s - 327us/step - accuracy: 0.1131 - loss: 658687.7500
Epoch 10/16
612/612 - 0s - 329us/step - accuracy: 0.1163 - loss: 812531.5000
Epoch 11/16
612/612 - 0s - 329us/step - accuracy: 0.1136 - loss: 974621.8125
Epoch 12/16
612/612 - 0s - 327us/step - accuracy: 0.1142 - loss: 1151674.3750
Epoch 13/16
612/612 - 0s - 319us/step - accuracy: 0.1143 - loss: 1338765.1250
Epoch 14/16

The acuuracy increases but it is still not satisfying.  The loss function doesn't converge. 

In [655]:
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step
Pred        BASEL  VALENTIA
True                       
BASEL        2977         4
BELGRADE      876         0
BUDAPEST      155         0
DEBILT         51         0
DUSSELDORF     34         0
HEATHROW       69         0
KASSEL          9         0
LJUBLJANA      59         0
MAASTRICHT      5         0
MADRID        339         0
MUNCHENB        5         0
OSLO            5         0
STOCKHOLM       1         0
VALENTIA        1         0


Only 2 labels are predicted.

In [659]:
# Second CNN Model with doubling batch size from 30 to 60.
epochs = 16
batch_size = 60
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [660]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [661]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/16
306/306 - 0s - 1ms/step - accuracy: 0.0975 - loss: 185.0072
Epoch 2/16
306/306 - 0s - 499us/step - accuracy: 0.0992 - loss: 1501.1897
Epoch 3/16
306/306 - 0s - 496us/step - accuracy: 0.1006 - loss: 5325.1831
Epoch 4/16
306/306 - 0s - 493us/step - accuracy: 0.1009 - loss: 12398.2939
Epoch 5/16
306/306 - 0s - 485us/step - accuracy: 0.1041 - loss: 22746.7715
Epoch 6/16
306/306 - 0s - 488us/step - accuracy: 0.1067 - loss: 37272.4023
Epoch 7/16
306/306 - 0s - 489us/step - accuracy: 0.1086 - loss: 56138.5703
Epoch 8/16
306/306 - 0s - 495us/step - accuracy: 0.1100 - loss: 79511.8984
Epoch 9/16
306/306 - 0s - 486us/step - accuracy: 0.1125 - loss: 107954.8125
Epoch 10/16
306/306 - 0s - 489us/step - accuracy: 0.1163 - loss: 141484.9531
Epoch 11/16
306/306 - 0s - 503us/step - accuracy: 0.1174 - loss: 179897.9688
Epoch 12/16
306/306 - 0s - 486us/step - accuracy: 0.1200 - loss: 224133.0781
Epoch 13/16
306/306 - 0s - 502us/step - accuracy: 0.1189 - loss: 273681.2188
Epoch 14/16
306/306 - 

The accuracy remains about 12% and loss function does not converge.

In [662]:
# Model 3 Increasing epochs and n_hidden
epochs = 32
batch_size = 60
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid'))

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [663]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [664]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.2030 - loss: 95.8864
Epoch 2/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - accuracy: 0.1387 - loss: 1764.1814
Epoch 3/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - accuracy: 0.1337 - loss: 6517.3486
Epoch 4/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - accuracy: 0.1320 - loss: 16068.2793
Epoch 5/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - accuracy: 0.1314 - loss: 30949.9121
Epoch 6/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 599us/step - accuracy: 0.1317 - loss: 51484.5703
Epoch 7/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - accuracy: 0.1325 - loss: 78234.5000
Epoch 8/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - accuracy: 0.1318 - loss: 111056.7266
Epoch 9/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - accuracy: 0.1311 - loss: 150073.2344
Epoch 10/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.1256 - loss: 195914.0938
Epoch 11/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step - accuracy: 0.1305 - loss:

The acuuracy still remains at 12%. The loss is increasing.

In [665]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step
Pred        BASEL  VALENTIA
True                       
BASEL        2977         4
BELGRADE      876         0
BUDAPEST      155         0
DEBILT         51         0
DUSSELDORF     34         0
HEATHROW       69         0
KASSEL          9         0
LJUBLJANA      59         0
MAASTRICHT      5         0
MADRID        339         0
MUNCHENB        5         0
OSLO            5         0
STOCKHOLM       1         0
VALENTIA        1         0


Only 2 lables are predicted. The model **fails to predict other 13 stations.**

In [666]:
# Model 4, Chaning final activation function from sigmoid to tanh
epochs = 32
batch_size = 60
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh'))

/opt/anaconda3/envs/notebook/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [667]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [668]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - accuracy: 0.0434 - loss: 25.6245
Epoch 2/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.2171 - loss: 23.2140
Epoch 3/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - accuracy: 0.2589 - loss: 29.4685
Epoch 4/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - accuracy: 0.2545 - loss: 29.7243
Epoch 5/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.2552 - loss: 29.5322
Epoch 6/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - accuracy: 0.2564 - loss: 27.5378
Epoch 7/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - accuracy: 0.2571 - loss: 20.3251
Epoch 8/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - accuracy: 0.2560 - loss: 20.3622
Epoch 9/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - accuracy: 0.2570 - loss: 20.4142
Epoch 10/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.2568 - loss: 20.4043
Epoch 11/32
306/306 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - accuracy: 0.2559 - loss: 20.4043
Epoch 12/32
306/306

The model accuracy improved but the loss function still doesn't converge.

In [669]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step
Pred        BASEL  BELGRADE  DUSSELDORF  MUNCHENB  SONNBLICK  STOCKHOLM  \
True                                                                      
BASEL        1331       210         109        74        128        891   
BELGRADE      765        34          27         5         33          9   
BUDAPEST      143         5           5         0          1          1   
DEBILT         51         0           0         0          0          0   
DUSSELDORF     33         1           0         0          0          0   
HEATHROW       69         0           0         0          0          0   
KASSEL          9         0           0         0          0          0   
LJUBLJANA      55         3           0         1          0          0   
MAASTRICHT      5         0           0         0          0          0   
MADRID        214        37          17         9         22         38   
MUNCHENB        5         0           0         0        

More stations are now labeled in predicted stations. But only Basel and Belgrade had few correct predictions. Other 13 stations were not predicted correctly at all.

#### Exporting Cleaned Dataset with Date column

In [90]:
# Adding Date column
weather['DATE'] = date_

/var/folders/xd/gd6kszh506xbhlz9mv23vpxh0000gn/T/ipykernel_49124/487308344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['DATE'] = date_


In [91]:
# Correcting the formating for date column
weather['DATE'] = pd.to_datetime(weather['DATE'], format = '%Y%m%d')

/var/folders/xd/gd6kszh506xbhlz9mv23vpxh0000gn/T/ipykernel_49124/741381049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['DATE'] = pd.to_datetime(weather['DATE'], format = '%Y%m%d')


In [92]:
weather.head(2)

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.0,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.1,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,1960-01-01
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.0,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.3,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,1960-01-02


In [93]:
# Exporting the weather dataset as a cleaned version
weather.to_csv(os.path.join(path, 'Unsupervised','weather_cleaned.csv'),index=False)